## Set Parameters

1. Set the H3 resolution to aggregate the results on.

    To see the average size of a hexagon at a given resolution go to the
    [official H3 documentation](https://h3geo.org/docs/core-library/restable). The currently set resolution 7 has on
    average an edge length of 1.2 km which can be freely interpreted as a radius.

In [ ]:
resolution = 7

2. These are the current 24 high-level categories for POIs:

    ```zsh
    administration, airport, apartment, art_culture, automobile, beauty, cafe, drinks, education, entertainment, food, groceries, medical, misc, office, public_service, public_transportation, recreation, religious_building, service, shopping, social_service, sport, tourism, wholesaler
    ```

In [ ]:
category = 'groceries'

3. Set per how many citizens you want to weight the POI count for a given category.

In [ ]:
per_x_citizens = 1000

## Send Queries

Create a Spark session that is used to connect to the Neo4j instance.

In [ ]:
from kuwala.modules.common import get_spark_session

sp = get_spark_session(memory_in_gb=16)

Get population per hexagon.

In [ ]:
from kuwala.modules.population_controller import get_population_in_h3

population = get_population_in_h3(sp=sp, resolution=resolution, polygon_coords=None)

population.select('h3_index', 'total').show(n=5)

Get number of POIs belonging to a selected category per hexagon.

In [ ]:
from kuwala.modules.poi_controller import get_pois_by_category_in_h3

pois = get_pois_by_category_in_h3(sp=sp, category=category, resolution=resolution, polygon_coords=None)

pois.show(n=5)

Calculate number of POIs per 1,000 citizens

In [ ]:
from pyspark.sql.functions import col

number_of_pois_in_category = f'number_of_{category}'
category_per_x = f'{category}_per_{per_x_citizens}'
population_in_x = f'population_in_{per_x_citizens}'

pois = pois.withColumnRenamed('h3_index', 'join_h3_index')
population = population \
    .withColumn(population_in_x, col('total') / per_x_citizens) \
    .select('h3_index', population_in_x)
result = population \
    .join(pois, population.h3_index == pois.join_h3_index, 'left') \
    .drop('join_h3_index') \
    .fillna(0, subset=[number_of_pois_in_category]) \
    .withColumn(category_per_x, col(number_of_pois_in_category) / col(population_in_x)) \
    .select('h3_index', category_per_x, number_of_pois_in_category, population_in_x)

result.show(n=5)

## Visualize results on map

In [ ]:
from unfolded.map_sdk import UnfoldedMap
from sidecar import Sidecar
from uuid import uuid4

unfolded_map = UnfoldedMap()
sc = Sidecar(title=f'{category} by population', anchor='split-right')

with sc:
    display(unfolded_map)

dataset_id=uuid4()

unfolded_map.add_dataset({
    'uuid': dataset_id,
    'label': f'{category} per thousand citizens',
    'data': result.toPandas()
})